In [4]:
import numpy as np
import pandas as pd
from os.path import join
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
data_path = '../../data/classification/old'

df = pd.read_csv(join(data_path, 'cal_now_values.csv'))
df1 = pd.read_csv(join(data_path, 'class1.csv'))
df2 = pd.read_csv(join(data_path, 'class2.csv'))
df3 = pd.read_csv(join(data_path, 'class3.csv'))
dict = {}
for name, code in zip(df1['行业名称'], df1['申万行业代码']):
    dict[code] = name
for name, code in zip(df2['行业名称'], df2['申万行业代码']):
    dict[code] = name
for name, code in zip(df3['行业名称'], df3['申万行业代码']):
    dict[code] = name

In [3]:
from stocks_info import StocksInfo
import tushare as ts
with open('tushare_token.txt', 'r') as f:
    tushare_token = f.readline()
ts.set_token(tushare_token)
tushare_api = ts.pro_api()
s = StocksInfo(api=tushare_api)
new_cols = [col for col in df.columns if col != 'TICKER_SYMBOL'] + ['TICKER_SYMBOL']
df = df[new_cols]
stock_name = []
stock_symbol = []
for i in df['TICKER_SYMBOL'].values:
    code = str(i).zfill(6)
    stock_symbol.append(code)
    if code in s.stock_name_dict:
        stock_name.append(s.stock_name_dict[code])
    else:
        stock_name.append('')
df['STOCK_NAME'] = stock_name
df['TICKER_SYMBOL'] = stock_symbol

日期:  20210522
时间:  06:40:26
----------------------------------------------------------------------
获取交易日历...
获取股票名称...
获取tushare可转债表...


In [4]:
data_path = '../../data/classification/shenwan'

sw1 = pd.read_csv(join(data_path, 'shenwan_l1.csv'))
sw2 = pd.read_csv(join(data_path, 'shenwan_l2.csv'))
sw3 = pd.read_csv(join(data_path, 'shenwan_l3.csv'))
dict_sw = {}
for name, code in zip(sw1['industryName'], sw1['industrySymbol']):
    dict_sw[code] = name
for name, code in zip(sw2['industryName'], sw2['industrySymbol']):
    dict_sw[code] = name
for name, code in zip(sw3['industryName'], sw3['industrySymbol']):
    dict_sw[code] = name

df_swc = pd.read_csv(join(data_path, 'shenwan_classification.csv'))
df_swc = df_swc.sort_values(by=['ticker','intoDate'])
dict_swc = {}
for i, row in df_swc.iterrows():
    if np.isnan(row['industrySymbol']):
        print(row['ticker'], row['secShortName'])
    else:
        dict_swc[row['ticker']] = (int(row['industrySymbol']), row['industryName3'])

# Test Shenwan Classes
***

In [5]:
set_1 = set()
set_2 = set()
set_3 = set()
L1 = list(df['classifi_by_14L1'].values.copy())
print(len(set(L1)))
for i, l in enumerate(L1):
    if l not in dict_sw:
        set_1.add(l)
    else:
        L1[i] = dict_sw[l]
L2 = list(df['classifi_by_14L2'].values.copy())
print(len(set(L2)))
for i, l in enumerate(L2):
    if l not in dict_sw:
        set_2.add(l)
    else:
        L2[i] = dict_sw[l]
L3 = list(df['classifi_by_14L3'].values.copy())
print(len(set(L3)))
for i, l in enumerate(L3):
    if l not in dict_sw:
        set_3.add(l)
    else:
        L3[i] = dict_sw[l]

28
104
242


In [6]:
[(s, dict[s]) for s in set_3]

[(310304, '航空航天设备'),
 (310402, '其他交运设备服务'),
 (270201, '元件'),
 (320101, '交换设备'),
 (240106, '小金属'),
 (260402, '电气自控设备'),
 (260404, '输变电设备'),
 (450101, '百货零售'),
 (450103, '商业物业经营'),
 (310104, '专用汽车'),
 (250201, '建筑施工'),
 (250202, '装饰园林'),
 (350201, '服装'),
 (470302, '软件开发及服务'),
 (470303, '系统集成')]

# Fixed
***

In [7]:
L1 = list(df['classifi_by_14L1'].values.copy())
print(len(set(L1)))
for i, l in enumerate(df['classifi_by_14L1'].values):
    L1[i] = dict[l]
df['14_L1_NAME'] = L1

L2 = list(df['classifi_by_14L2'].values.copy())
print(len(set(L2)))
for i, l in enumerate(df['classifi_by_14L2'].values):
    L2[i] = dict[l]
df['14_L2_NAME'] = L2

L3 = list(df['classifi_by_14L3'].values.copy())
L3_name = []
orig_L3 = []
orig_L3_name = []
fixed_flag = []
print(len(set(L3)))

for i, l in enumerate(df['classifi_by_14L3'].values):
    if l not in dict_sw:
        L3[i] = dict_swc[stock_symbol[i]][0]
        L3_name.append(dict_swc[stock_symbol[i]][1])
        orig_L3.append(l)
        orig_L3_name.append(dict[l])
        fixed_flag.append(1)
    else:
        L3_name.append(dict[l])
        orig_L3.append(np.nan)
        orig_L3_name.append('')
        fixed_flag.append(0)
df['classifi_by_14L3'] = L3
df['14_L3_NAME'] = L3_name
df['FIXED_FLAG'] = fixed_flag
df['ORIGINAL_L3'] = orig_L3
df['ORIGINAL_L3_NAME'] = orig_L3_name

print(len(set(df['classifi_by_14L3'].values)))
df = df.rename(columns={'year': 'YEAR', 
                        'month': 'MONTH', 
                        'classifi_by_14L1': '14_L1', 
                        'classifi_by_14L2': '14_L2', 
                        'classifi_by_14L3': '14_L3'})

28
104
242
227


In [8]:
df_new = df[['TICKER_SYMBOL', 'STOCK_NAME', 'YEAR', 'MONTH', 'REPORT_TYPE', 
             '14_L1', '14_L1_NAME', '14_L2', '14_L2_NAME', '14_L3', '14_L3_NAME',
             'FIXED_FLAG', 'ORIGINAL_L3', 'ORIGINAL_L3_NAME']]

data_path = '../../data/classification/'

df_new.to_excel(join(data_path, 'classification_fixed.xlsx'), index=False)
df_new.to_csv(join(data_path, 'classification_fixed.csv'), index=False)
# df.to_excel(join(data_path, 'total_fixed.xlsx'))

# Original + Shenwan L3 + Errors
***

In [9]:
L1 = list(df['classifi_by_14L1'].values.copy())
print(len(set(L1)))
for i, l in enumerate(L1):
    L1[i] = dict[l]
df['14L1_CN'] = L1

L2 = list(df['classifi_by_14L2'].values.copy())
print(len(set(L2)))
for i, l in enumerate(L2):
    L2[i] = dict[l]
df['14L2_CN'] = L2

L3 = list(df['classifi_by_14L3'].values.copy())
print(len(set(L3)))

ShenwanL3 = []
ShenwanL3_CN = []
for i, l in enumerate(L3):
    ShenwanL3.append(dict_swc[stock_symbol[i]][0])
    ShenwanL3_CN.append(dict_swc[stock_symbol[i]][1])
    L3[i] = dict[l]
df['14L3_CN'] = L3
df['ShenwanL3'] = ShenwanL3
df['ShenwanL3_CN'] = ShenwanL3_CN

28
104
242


In [10]:
df_new = df[['TICKER_SYMBOL', 'STOCK_NAME', 'year', 'month', 
             'REPORT_TYPE', 
             'classifi_by_14L1', '14L1_CN',
             'classifi_by_14L2', '14L2_CN',
             'classifi_by_14L3', '14L3_CN', 
             'ShenwanL3', 'ShenwanL3_CN']]

In [11]:
df_new.to_excel('classification.xlsx')
# df.to_excel('total.xlsx')

In [ ]:
lll = []
df_n = pd.DataFrame()
for i, row in df_new.iterrows():
    if row['classifi_by_14L3'] not in dict_sw:
        df_n = df_n.append(row)
df_n=df_n[['TICKER_SYMBOL', 'STOCK_NAME', 'year', 'month',
            'classifi_by_14L1', '14L1_CN',
             'classifi_by_14L2', '14L2_CN',
             'classifi_by_14L3', '14L3_CN', 
             'ShenwanL3', 'ShenwanL3_CN']]
df_n.head()

In [11]:
df_n.to_excel('errors.xlsx')

# Test Class
***

In [12]:
df1.to_excel('Class1.xlsx')
df2.to_excel('Class2.xlsx')
df3.to_excel('Class3.xlsx')

In [13]:
len(set(list(df['classifi_by_14L3'])))

242

In [21]:
len(set(df['industrySymbol_level_3'].values))

312

# Generate fixed classification

In [1]:
import numpy as np
import pandas as pd
from os.path import join
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
data_path = '../../data/classification'

df = pd.read_csv(join(data_path, 'old/cal_now_values.csv'))
df['TICKER_SYMBOL'] = df['TICKER_SYMBOL'].apply(lambda x: str(x).zfill(6))

# df1 = pd.read_csv(join(data_path, 'shenwan/class1.csv'))
# df2 = pd.read_csv(join(data_path, 'shenwan/class2.csv'))
# dict_name = {}

# for name, code in zip(df1['行业名称'], df1['申万行业代码']):
#     dict_name[code] = name
# for name, code in zip(df2['行业名称'], df2['申万行业代码']):
#     dict_name[code] = name

df1 = pd.read_csv(
    join(data_path, 'shenwan/shenwan_l1.csv'))
df2 = pd.read_csv(
    join(data_path, 'shenwan/shenwan_l2.csv'))
sw_name_dict = {}
for code, name in zip(df1['industrySymbol'], df1['industryName']):
    sw_name_dict[code] = name
for code, name in zip(df2['industrySymbol'], df2['industryName']):
    sw_name_dict[code] = name

df_fixed = pd.read_csv(join(data_path, 'classification_fixed_L2.csv'))
df_fixed['ticker'] = df_fixed['ticker'].apply(lambda x: str(x).zfill(6))

In [4]:
L1 = list(df['classifi_by_14L1'].values.copy())
print(len(set(L1)))
for i, l in enumerate(L1):
    L1[i] = sw_name_dict[l]
df['class_L1_CN'] = L1

L2 = list(df['classifi_by_14L2'].values.copy())
print(len(set(L2)))
for i, l in enumerate(L2):
    L2[i] = sw_name_dict[l]
df['class_L2_CN'] = L2

28
104


In [5]:
df= df[['TICKER_SYMBOL', 'classifi_by_14L1', 'class_L1_CN', 'classifi_by_14L2', 'class_L2_CN', 'END_DATE']]
df = df.rename(columns={
    'TICKER_SYMBOL': 'ticker',
    'END_DATE': 'endDate', 
    'classifi_by_14L1': 'class_L1',
    'classifi_by_14L2': 'class_L2'
}, errors='ignore')
df = df.reset_index()

In [6]:
df = df.set_index(['ticker', 'endDate'])
df = df.drop(columns=['index'])
month_2_date = {
    3: '-03-31',
    6: '-06-30',
    9: '-09-30',
    12: '-12-31'
}
for _, row in df_fixed.iterrows():
    ticker = row['ticker']
    month = row['month']
    class_L2 = str(row['fixed_L2'])
    class_L2_CN = row['fixed_L2_CN']
    class_L1 = class_L2[:2] +'0000'
    class_L1_CN = sw_name_dict[int(class_L1)]
    end_date = str(row['year']) + month_2_date[row['month']]
    df.loc[(ticker, end_date), 'class_L1'] = class_L1
    df.loc[(ticker, end_date), 'class_L1_CN'] = class_L1_CN
    df.loc[(ticker, end_date), 'class_L2'] = class_L2
    df.loc[(ticker, end_date), 'class_L2_CN'] = class_L2_CN

In [7]:
df = df.rename(columns={
    'TICKER_SYMBOL': 'ticker',
    'END_DATE': 'endDate', 
    'classifi_by_14L1': 'class_L1',
    'classifi_by_14L2': 'class_L2'
}, errors='ignore')
df = df.reset_index()


In [8]:
df.head()

,ticker,endDate,class_L1,class_L1_CN,class_L2,class_L2_CN
0,000001,2009-03-31,480000,银行,480100,银行
1,000001,2009-06-30,480000,银行,480100,银行
2,000001,2009-09-30,480000,银行,480100,银行
3,000001,2009-12-31,480000,银行,480100,银行
4,000001,2010-03-31,480000,银行,480100,银行


In [9]:
df.to_csv('../../data/classification/classification.csv', index=False)

In [ ]:
df = df[:10000]

In [3]:
selected_1 = df.groupby(['TICKER_SYMBOL', 'END_DATE']).apply(
    lambda x: tuple(x[x['END_DATE_REP'] == x['END_DATE_REP'].values[-1]].index))

In [4]:
idx_dup_1 = []
for i in selected_1.values:
    if len(i) > 1:
        idx_dup_1.extend(i)

In [5]:
df.loc[idx_dup_1][['TICKER_SYMBOL', 'END_DATE', 'END_DATE_REP', 'PUBLISH_DATE']]

,TICKER_SYMBOL,END_DATE,END_DATE_REP,PUBLISH_DATE


In [6]:
selected_2 = df.groupby(['TICKER_SYMBOL', 'END_DATE', 'END_DATE_REP']).apply(lambda x: tuple(x[x['PUBLISH_DATE'] == x['PUBLISH_DATE'].values[-1]].index))
idx_dup_2 = []
for i in selected_2.values:
    if len(i) > 1:
        idx_dup_2.extend(i)
df.loc[idx_dup_2][['TICKER_SYMBOL', 'END_DATE', 'END_DATE_REP', 'PUBLISH_DATE']]

,TICKER_SYMBOL,END_DATE,END_DATE_REP,PUBLISH_DATE
